In [1]:
import redis
import json

In [2]:
REDIS = redis.Redis(host='data_store')

In [10]:
#Test
REDIS.set('my_incrementer', 42)

True

In [3]:
#Manager needs to do many things. One of which is manage the the company list and populate the data-store
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }
REDIS.set('companies', json.dumps(companies))

True

In [5]:
REDIS.get('IEX_Stock_Count')

b'38'